In [1]:
import numpy as np
import random
import time
import operator

In [2]:
GL_MUTATION_RATE = 30
GL_POPULATION_SIZE = 1500
GL_FITNESS_THRESHOLD = 3

In [3]:
class Genome():
    chromosomes = []
    fitness = 99

In [4]:
class GenAlgoOf8QueenProbSolver():
    def __init__(self):
        self.population = []
        self.generation = 0
        self.result = None

    def CreateInitPopulation(self, size):
        for x in range(size):
            newGenome = Genome()
            newGenome.chromosomes = random.sample(range(0, 8), 8)
            newGenome.fitness = self.Evaluate(newGenome.chromosomes)
            self.population.append(newGenome)

    def BuildTheChessBoard(self, chromo):
        chessboard = np.zeros((8, 8), dtype=int)
        for i in range(8):
            y = chromo[i]
            chessboard[y][i] = 1
        return chessboard

    
    def Evaluate(self, chromosomes):
        chessboard = self.BuildTheChessBoard(chromosomes)
        calculated_fitness = 0

        # Horizontal Control
        for i in range(8):
            count = np.count_nonzero(chessboard[i])
            if count > 1:
                calculated_fitness += count - 1

        # Cross Control
        for y in range(8):
            count = 0
            for x in range(8):
                if chessboard[x + y, x] == 1:
                    count += 1
                if (x + y) >= 7:
                    break
            if count > 1:
                calculated_fitness += count - 1

        for y in range(1, 8):
            count = 0
            for x in range(8):
                if chessboard[x, x + y] == 1:
                    count += 1
                if (x + y) >= 7:
                    break
            if count > 1:
                calculated_fitness += count - 1

        for y in range(0, 8):
            count = 0
            for x in range(0, 8):
                if chessboard[y + x, 7 - x] == 1:
                    count += 1
                if (y + x) >= 7:
                    break
            if count > 1:
                calculated_fitness += count - 1

        for y in range(0, 8):
            count = 0
            for x in range(0, 7 - y):
                if chessboard[x, (6 - y) - x] == 1:
                    count += 1
                if (x) >= 7:
                    break
            if count > 1:
                calculated_fitness += count - 1

        return calculated_fitness

    def RouletteWheelSelection(self):
        max = sum(genom.fitness for genom in self.population)
        pick = random.uniform(0, max)
        current = 0
        for genom in self.population:
            current += genom.fitness
            if current > pick:
                return genom

    def SinglePointCrossover(self):
        parent1 = self.RouletteWheelSelection().chromosomes
        parent2 = self.population[random.randrange(0, len(self.population))].chromosomes

        point = random.randint(2, 6)
        child = Genome()
        child.chromosomes = parent1[0:point]
        child.chromosomes.extend(parent2[point:8])
        child.fitness = self.Evaluate(child.chromosomes)

        if random.randrange(0, 100) < GL_MUTATION_RATE:
            self.Mutation(child.chromosomes)
        else:
            self.population.append(child)

    def Mutation(self, chromo):
        chromo[random.randrange(0, 8)] = random.randrange(0, 8)
        newGenome = Genome()
        newGenome.chromosomes = chromo
        newGenome.fitness = self.Evaluate(newGenome.chromosomes)
        self.population.append(newGenome)

    def RemoveWeakness(self):
        for genom in self.population:
            if genom.fitness > GL_FITNESS_THRESHOLD:
                self.population.remove(genom)

    def ControlResult(self):
        for genom in self.population:
            if(genom.fitness < 1):
                self.result = genom.chromosomes

    def Start(self):
        # if result not found - reset and try again
        while self.result is None:
            self.population = []
            self.generation = 0

            self.CreateInitPopulation(GL_POPULATION_SIZE)
            print("--:New Population Created:--")
            while self.generation < 30 and self.result is None:

                for x in range(int(len(self.population) / 2)):
                    self.SinglePointCrossover()

                self.RemoveWeakness()
                self.ControlResult()
                self.generation += 1
                print("POPULATION =", len(self.population), "GENERATION =", self.generation)

        print(self.result)


In [5]:
GenAlgoOf8QueenProbSolver().Start()

--:New Population Created:--
POPULATION = 1292 GENERATION = 1
[3, 6, 4, 1, 5, 0, 2, 7]


In [6]:
import requests
url='https://lf8q0kx152.execute-api.us-east-2.amazonaws.com/default/computeFitnessScore'
x=requests.post(url,json={"qconfig": "3 6 4 1 5 0 2 7","userID":277541,"githubLink":"https://github.com/enthusiastic-owl/QvsQ"})
print(x.text)

{"No Of Attempts lapsed out of 3": 1, "submittedConfiguration": [3, 6, 4, 1, 5, 0, 2, 7], "configurationStatus": "Valid", "configurationScore": 100.0}
